In [2]:
# Preprocessing
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

# Model Classifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Model selection
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, balanced_accuracy_score, roc_auc_score, confusion_matrix
from sklearn.neural_network import MLPClassifier
# from sklearn.metrics import (precision_recall_curve, auc, roc_curve, recall_score, classification_report, precision_recall_fscore_support)

# Model persistence
import pickle

# Others
import warnings
import gc

# Visualization
import seaborn as sns
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt

/Users/ziyuanhan/anaconda3/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import warnings
from random import sample
# warnings.filterwarnings(action='once')
warnings.filterwarnings('ignore')

%matplotlib inline

# Load test data and feature engineer it

In [4]:
final_test = pd.read_csv('data/one_time_test2m.csv', parse_dates = ['click_time'])


In [5]:
def fea_eng(Y):
    X = Y
    X['day'] = X['click_time'].dt.day
    X['hour'] = X['click_time'].dt.hour
    X['minute'] = X['click_time'].dt.minute
    X['second'] = X['click_time'].dt.second
    heads = X.columns
    print(heads)
    for i in list(range(0,5)) + [6]:
        X[heads[i]] = X[heads[i]].astype('category')
    return X

In [6]:
final = fea_eng(final_test)

Index(['ip', 'app', 'device', 'os', 'channel', 'click_time', 'attributed_time',
       'is_attributed', 'day', 'hour', 'minute', 'second'],
      dtype='object')


In [7]:
# down sample the train set
df_majority = final[final.is_attributed==0]
df_minority = final[final.is_attributed==1]

df_majority_downsampled = df_majority.sample(replace=False,    # sample without replacement
                                frac=len(df_minority)/len(df_majority),     
                                random_state=42) # reproducible results

balanced = pd.concat([df_majority_downsampled, df_minority])
balanced = balanced.sample(frac=1)

# balanced = balanced.drop(columns=['click_time','attributed_time'])
balanced = final.drop(columns=['click_time','attributed_time'])
balanced['is_attributed'] = balanced['is_attributed'].astype('int64')

In [8]:
balanced.columns

Index(['ip', 'app', 'device', 'os', 'channel', 'is_attributed', 'day', 'hour',
       'minute', 'second'],
      dtype='object')

In [39]:
# balanced_train.columns

In [40]:
test_fe = balanced

In [41]:
test_fe[['day', 'hour', 'minute', 'second']] = test_fe[['day', 'hour', 'minute', 'second']].astype('float')
test_fe[['ip', 'app', 'device', 'os', 'channel']] = test_fe[['ip', 'app', 'device', 'os', 'channel']].astype('category')

In [42]:
# Define all the groupby transformations
groupby_aggregations = [
    
    # V1 - GroupBy Features #
    #########################    
    # Variance in day, for ip-app-channel
    {'groupby': ['ip','app','channel'], 'select': 'day', 'agg': 'var'},
    # Variance in hour, for ip-app-os
    {'groupby': ['ip','app','os'], 'select': 'hour', 'agg': 'var'},
    # Count, for ip-day-hour
    {'groupby': ['ip','day','hour'], 'select': 'channel', 'agg': 'count'},
    # Count, for ip-app
    {'groupby': ['ip', 'app'], 'select': 'channel', 'agg': 'count'},        
    # Count, for ip-app-os
    {'groupby': ['ip', 'app', 'os'], 'select': 'channel', 'agg': 'count'},
    # Count, for ip-app-day-hour
    {'groupby': ['ip','app','day','hour'], 'select': 'channel', 'agg': 'count'},
    # Mean hour, for ip-app-channel
    {'groupby': ['ip','app','channel'], 'select': 'hour', 'agg': 'mean'}, 
    
    # V2 - GroupBy Features #
    #########################
    # Average clicks on app by distinct users; is it an app they return to?
    {'groupby': ['app'], 
     'select': 'ip', 
     'agg': lambda x: np.divide(float(len(x)),float(len(x.unique()))), 
     'agg_name': 'AvgViewPerDistinct'
    },
    # How popular is the app or channel?
    {'groupby': ['app'], 'select': 'channel', 'agg': 'count'},
    {'groupby': ['channel'], 'select': 'app', 'agg': 'count'},
    
    # V3 - GroupBy Features                                              #
    # https://www.kaggle.com/bk0000/non-blending-lightgbm-model-lb-0-977 #
    ###################################################################### 
    {'groupby': ['ip'], 'select': 'channel', 'agg': 'nunique'}, 
    {'groupby': ['ip'], 'select': 'app', 'agg': 'nunique'}, 
    {'groupby': ['ip','day'], 'select': 'hour', 'agg': 'nunique'}, 
    {'groupby': ['ip','app'], 'select': 'os', 'agg': 'nunique'}, 
    {'groupby': ['ip'], 'select': 'device', 'agg': 'nunique'}, 
    {'groupby': ['app'], 'select': 'channel', 'agg': 'nunique'}, 
    {'groupby': ['ip', 'device', 'os'], 'select': 'app', 'agg': 'nunique'}, 
    {'groupby': ['ip','device','os'], 'select': 'app', 'agg': 'cumcount'}, 
    {'groupby': ['ip'], 'select': 'app', 'agg': 'cumcount'}, 
    {'groupby': ['ip'], 'select': 'os', 'agg': 'cumcount'}, 
    {'groupby': ['ip','day','channel'], 'select': 'hour', 'agg': 'var'}    
]



new_features =[]
# Apply all the groupby transformations
for spec in groupby_aggregations:
    
    # Name of the aggregation we're applying
    agg_name = spec['agg_name'] if 'agg_name' in spec else spec['agg']
    
    # Name of new feature
    new_feature = '{}_{}_{}'.format('_'.join(spec['groupby']), agg_name, spec['select'])
    new_features.append(new_feature)
    
    # Info
    print("Grouping by {}, and aggregating {} with {}".format(
        spec['groupby'], spec['select'], agg_name
    ))
    
    # Unique list of features to select
    all_features = list(set(spec['groupby'] + [spec['select']]))
    
    # Perform the groupby
    gp = test_fe[all_features]. \
        groupby(spec['groupby'])[spec['select']]. \
        agg(spec['agg']). \
        reset_index(). \
        rename(index=str, columns={spec['select']: new_feature})
        
    # Merge back to X_total
    if 'cumcount' == spec['agg']:
        test_fe[new_feature] = gp[0].values
    else:
        # train_fe[spec['groupby']] = train_fe[spec['groupby']].astype('category')
        test_fe = test_fe.merge(gp, on=spec['groupby'], how='left')
        
     # Clear memory
    del gp
    gc.collect()

test_fe.head()

Grouping by ['ip', 'app', 'channel'], and aggregating day with var
Grouping by ['ip', 'app', 'os'], and aggregating hour with var
Grouping by ['ip', 'day', 'hour'], and aggregating channel with count
Grouping by ['ip', 'app'], and aggregating channel with count
Grouping by ['ip', 'app', 'os'], and aggregating channel with count
Grouping by ['ip', 'app', 'day', 'hour'], and aggregating channel with count
Grouping by ['ip', 'app', 'channel'], and aggregating hour with mean
Grouping by ['app'], and aggregating ip with AvgViewPerDistinct
Grouping by ['app'], and aggregating channel with count
Grouping by ['channel'], and aggregating app with count
Grouping by ['ip'], and aggregating channel with nunique
Grouping by ['ip'], and aggregating app with nunique
Grouping by ['ip', 'day'], and aggregating hour with nunique
Grouping by ['ip', 'app'], and aggregating os with nunique
Grouping by ['ip'], and aggregating device with nunique
Grouping by ['app'], and aggregating channel with nunique
Grou

,ip,app,device,os,channel,is_attributed,day,hour,minute,second,...,ip_nunique_app,ip_day_nunique_hour,ip_app_nunique_os,ip_nunique_device,app_nunique_channel,ip_device_os_nunique_app,ip_device_os_cumcount_app,ip_cumcount_app,ip_cumcount_os,ip_day_channel_var_hour
0,32069,9,1,13,134,0,9.0,14.0,14.0,56.0,...,5,4,2,1,38,3,0,0,0,NaN
1,147012,29,1,19,101,0,7.0,14.0,14.0,5.0,...,1,1,1,1,16,1,0,0,0,NaN
2,84896,11,1,13,469,0,8.0,9.0,51.0,43.0,...,46,24,20,9,15,25,0,0,0,25.563636
3,55957,14,1,13,467,0,9.0,14.0,58.0,10.0,...,23,14,9,2,33,18,0,0,0,4.500000
4,847,12,1,22,245,0,8.0,9.0,54.0,18.0,...,11,7,4,2,30,4,0,0,0,13.766667


In [43]:
test_fe['ip_app_channel_var_day'] = test_fe['ip_app_channel_var_day'].fillna(0) 
test_fe['ip_app_os_var_hour'] = test_fe['ip_app_os_var_hour'].fillna(0) 
test_fe['ip_day_channel_var_hour'] = test_fe['ip_day_channel_var_hour'].fillna(0)
# test_fe = test_fe.fillna(0)

In [44]:
test_fe.head()

,ip,app,device,os,channel,is_attributed,day,hour,minute,second,...,ip_nunique_app,ip_day_nunique_hour,ip_app_nunique_os,ip_nunique_device,app_nunique_channel,ip_device_os_nunique_app,ip_device_os_cumcount_app,ip_cumcount_app,ip_cumcount_os,ip_day_channel_var_hour
0,32069,9,1,13,134,0,9.0,14.0,14.0,56.0,...,5,4,2,1,38,3,0,0,0,0.000000
1,147012,29,1,19,101,0,7.0,14.0,14.0,5.0,...,1,1,1,1,16,1,0,0,0,0.000000
2,84896,11,1,13,469,0,8.0,9.0,51.0,43.0,...,46,24,20,9,15,25,0,0,0,25.563636
3,55957,14,1,13,467,0,9.0,14.0,58.0,10.0,...,23,14,9,2,33,18,0,0,0,4.500000
4,847,12,1,22,245,0,8.0,9.0,54.0,18.0,...,11,7,4,2,30,4,0,0,0,13.766667


In [45]:
x_full_train.columns

Index(['ip', 'app', 'device', 'os', 'channel', 'day', 'hour', 'minute',
       'second', 'ip_app_channel_var_day', 'ip_app_os_var_hour',
       'ip_day_hour_count_channel', 'ip_app_count_channel',
       'ip_app_os_count_channel', 'ip_app_day_hour_count_channel',
       'ip_app_channel_mean_hour', 'app_AvgViewPerDistinct_ip',
       'app_count_channel', 'channel_count_app', 'ip_nunique_channel',
       'ip_nunique_app', 'ip_day_nunique_hour', 'ip_app_nunique_os',
       'ip_nunique_device', 'app_nunique_channel', 'ip_device_os_nunique_app',
       'ip_device_os_cumcount_app', 'ip_cumcount_app', 'ip_cumcount_os',
       'ip_day_channel_var_hour'],
      dtype='object')

In [46]:
X = test_fe.drop(['is_attributed'],axis=1)
y = test_fe[['is_attributed']]
X.columns

Index(['ip', 'app', 'device', 'os', 'channel', 'day', 'hour', 'minute',
       'second', 'ip_app_channel_var_day', 'ip_app_os_var_hour',
       'ip_day_hour_count_channel', 'ip_app_count_channel',
       'ip_app_os_count_channel', 'ip_app_day_hour_count_channel',
       'ip_app_channel_mean_hour', 'app_AvgViewPerDistinct_ip',
       'app_count_channel', 'channel_count_app', 'ip_nunique_channel',
       'ip_nunique_app', 'ip_day_nunique_hour', 'ip_app_nunique_os',
       'ip_nunique_device', 'app_nunique_channel', 'ip_device_os_nunique_app',
       'ip_device_os_cumcount_app', 'ip_cumcount_app', 'ip_cumcount_os',
       'ip_day_channel_var_hour'],
      dtype='object')

# Load training data and feature engineer it

In [10]:
talking = pd.read_csv('./data/param_tuning18m.csv') 
talking['click_time'] = pd.to_datetime(talking['click_time'], errors='coerce')

In [11]:
# transform click_time into day, hour, minute and second
talking['day'] = talking['click_time'].dt.day
talking['hour'] = talking['click_time'].dt.hour
talking['minute'] = talking['click_time'].dt.minute
talking['second'] = talking['click_time'].dt.second
talking.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,day,hour,minute,second
0,83604,64,1,13,459,2017-11-08 07:18:18,NaN,0,8,7,18,18
1,28021,3,1,13,409,2017-11-07 03:54:37,NaN,0,7,3,54,37
2,44456,3,1,15,480,2017-11-09 12:39:05,NaN,0,9,12,39,5
3,134076,9,1,13,258,2017-11-07 10:55:26,NaN,0,7,10,55,26
4,138561,6,1,13,459,2017-11-08 03:33:21,NaN,0,8,3,33,21


In [12]:
# ip, app, device, os, channel, is_attributed and the time features we just created
# are encoded to be integers, we tranform them into categorical data
heads = talking.columns
print(heads)
for i in list(range(0,5)) + [6]:
    talking[heads[i]] = talking[heads[i]].astype('category')
talking.info()

Index(['ip', 'app', 'device', 'os', 'channel', 'click_time', 'attributed_time',
       'is_attributed', 'day', 'hour', 'minute', 'second'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18000000 entries, 0 to 17999999
Data columns (total 12 columns):
ip                 category
app                category
device             category
os                 category
channel            category
click_time         datetime64[ns]
attributed_time    category
is_attributed      int64
day                int64
hour               int64
minute             int64
second             int64
dtypes: category(6), datetime64[ns](1), int64(5)
memory usage: 1.1 GB


In [13]:
# down sample the train set
df_majority = talking[talking.is_attributed==0]
df_minority = talking[talking.is_attributed==1]

df_majority_downsampled = df_majority.sample(replace=False,    # sample without replacement
                                frac=len(df_minority)/len(df_majority),     
                                random_state=42) # reproducible results

balanced_train = pd.concat([df_majority_downsampled, df_minority])
balanced_train = balanced_train.sample(frac=1)

balanced_train = balanced_train.drop(columns=['click_time','attributed_time'])
balanced_train['is_attributed'] = balanced_train['is_attributed'].astype('int64')

In [14]:
# split 20m into 2m train data and 18m validation data.
train = balanced_train.sample(frac=0.1, replace=False, random_state = 42)
valid = balanced_train.drop(train.index)

In [15]:
# X_full_train = train.drop(['is_attributed'],axis=1)
# y_full_train = train[['is_attributed']]
# X_full_test= valid.drop(['is_attributed'],axis=1)
# y_full_test = valid[['is_attributed']]

In [16]:
train_fe = train

In [17]:
train_fe[['day', 'hour', 'minute', 'second']] = train_fe[['day', 'hour', 'minute', 'second']].astype('float')
train_fe[['ip', 'app', 'device', 'os', 'channel']] = train_fe[['ip', 'app', 'device', 'os', 'channel']].astype('category')

In [18]:
# Define all the groupby transformations
groupby_aggregations = [
    
    # V1 - GroupBy Features #
    #########################    
    # Variance in day, for ip-app-channel
    {'groupby': ['ip','app','channel'], 'select': 'day', 'agg': 'var'},
    # Variance in hour, for ip-app-os
    {'groupby': ['ip','app','os'], 'select': 'hour', 'agg': 'var'},
    # Count, for ip-day-hour
    {'groupby': ['ip','day','hour'], 'select': 'channel', 'agg': 'count'},
    # Count, for ip-app
    {'groupby': ['ip', 'app'], 'select': 'channel', 'agg': 'count'},        
    # Count, for ip-app-os
    {'groupby': ['ip', 'app', 'os'], 'select': 'channel', 'agg': 'count'},
    # Count, for ip-app-day-hour
    {'groupby': ['ip','app','day','hour'], 'select': 'channel', 'agg': 'count'},
    # Mean hour, for ip-app-channel
    {'groupby': ['ip','app','channel'], 'select': 'hour', 'agg': 'mean'}, 
    
    # V2 - GroupBy Features #
    #########################
    # Average clicks on app by distinct users; is it an app they return to?
    {'groupby': ['app'], 
     'select': 'ip', 
     'agg': lambda x: np.divide(float(len(x)),float(len(x.unique()))), 
     'agg_name': 'AvgViewPerDistinct'
    },
    # How popular is the app or channel?
    {'groupby': ['app'], 'select': 'channel', 'agg': 'count'},
    {'groupby': ['channel'], 'select': 'app', 'agg': 'count'},
    
    # V3 - GroupBy Features                                              #
    # https://www.kaggle.com/bk0000/non-blending-lightgbm-model-lb-0-977 #
    ###################################################################### 
    {'groupby': ['ip'], 'select': 'channel', 'agg': 'nunique'}, 
    {'groupby': ['ip'], 'select': 'app', 'agg': 'nunique'}, 
    {'groupby': ['ip','day'], 'select': 'hour', 'agg': 'nunique'}, 
    {'groupby': ['ip','app'], 'select': 'os', 'agg': 'nunique'}, 
    {'groupby': ['ip'], 'select': 'device', 'agg': 'nunique'}, 
    {'groupby': ['app'], 'select': 'channel', 'agg': 'nunique'}, 
    {'groupby': ['ip', 'device', 'os'], 'select': 'app', 'agg': 'nunique'}, 
    {'groupby': ['ip','device','os'], 'select': 'app', 'agg': 'cumcount'}, 
    {'groupby': ['ip'], 'select': 'app', 'agg': 'cumcount'}, 
    {'groupby': ['ip'], 'select': 'os', 'agg': 'cumcount'}, 
    {'groupby': ['ip','day','channel'], 'select': 'hour', 'agg': 'var'}    
]



new_features =[]
# Apply all the groupby transformations
for spec in groupby_aggregations:
    
    # Name of the aggregation we're applying
    agg_name = spec['agg_name'] if 'agg_name' in spec else spec['agg']
    
    # Name of new feature
    new_feature = '{}_{}_{}'.format('_'.join(spec['groupby']), agg_name, spec['select'])
    new_features.append(new_feature)
    
    # Info
    print("Grouping by {}, and aggregating {} with {}".format(
        spec['groupby'], spec['select'], agg_name
    ))
    
    # Unique list of features to select
    all_features = list(set(spec['groupby'] + [spec['select']]))
    
    # Perform the groupby
    gp = train_fe[all_features]. \
        groupby(spec['groupby'])[spec['select']]. \
        agg(spec['agg']). \
        reset_index(). \
        rename(index=str, columns={spec['select']: new_feature})
        
    # Merge back to X_total
    if 'cumcount' == spec['agg']:
        train_fe[new_feature] = gp[0].values
    else:
        # train_fe[spec['groupby']] = train_fe[spec['groupby']].astype('category')
        train_fe = train_fe.merge(gp, on=spec['groupby'], how='left')
        
     # Clear memory
    del gp
    gc.collect()

train_fe.head()

Grouping by ['ip', 'app', 'channel'], and aggregating day with var
Grouping by ['ip', 'app', 'os'], and aggregating hour with var
Grouping by ['ip', 'day', 'hour'], and aggregating channel with count
Grouping by ['ip', 'app'], and aggregating channel with count
Grouping by ['ip', 'app', 'os'], and aggregating channel with count
Grouping by ['ip', 'app', 'day', 'hour'], and aggregating channel with count
Grouping by ['ip', 'app', 'channel'], and aggregating hour with mean
Grouping by ['app'], and aggregating ip with AvgViewPerDistinct
Grouping by ['app'], and aggregating channel with count
Grouping by ['channel'], and aggregating app with count
Grouping by ['ip'], and aggregating channel with nunique
Grouping by ['ip'], and aggregating app with nunique
Grouping by ['ip', 'day'], and aggregating hour with nunique
Grouping by ['ip', 'app'], and aggregating os with nunique
Grouping by ['ip'], and aggregating device with nunique
Grouping by ['app'], and aggregating channel with nunique
Grou

,ip,app,device,os,channel,is_attributed,day,hour,minute,second,...,ip_nunique_app,ip_day_nunique_hour,ip_app_nunique_os,ip_nunique_device,app_nunique_channel,ip_device_os_nunique_app,ip_device_os_cumcount_app,ip_cumcount_app,ip_cumcount_os,ip_day_channel_var_hour
0,326487,19,0,24,213,1,8.0,16.0,0.0,21.0,...,1,1,1,1,8,1,0,0,0,NaN
1,114414,2,2,9,477,0,7.0,1.0,51.0,8.0,...,1,1,1,1,20,1,0,0,0,NaN
2,86767,20,1,10,478,0,8.0,4.0,32.0,6.0,...,6,2,1,2,2,1,0,0,0,NaN
3,282934,29,1,64,343,0,8.0,4.0,42.0,32.0,...,1,1,1,1,8,1,0,0,0,NaN
4,121384,3,1,19,130,0,9.0,14.0,47.0,32.0,...,1,1,1,1,31,1,0,0,0,NaN


In [19]:
base_features = ['ip', 'app', 'device', 'os', 'channel']

In [20]:
train_fe['ip_app_channel_var_day'] = train_fe['ip_app_channel_var_day'].fillna(0) 
train_fe['ip_app_os_var_hour'] = train_fe['ip_app_os_var_hour'].fillna(0) 
train_fe['ip_day_channel_var_hour'] = train_fe['ip_day_channel_var_hour'].fillna(0)

In [21]:
train_gbc = train_fe[['channel','ip','channel_count_app','app','app_count_channel','is_attributed']]
# valid_gbc = train_fe[['channel','ip','channel_count_app','app','app_count_channel','is_attributed']]
x_full_train, x_full_test, y_full_train, y_full_test = train_test_split(train_fe.drop(['is_attributed'],axis=1),train_fe['is_attributed'], test_size=0.2, random_state=42)
x_reduced_train, x_reduced_test, y_reduced_train, y_reduced_test = train_test_split(train_gbc.drop(['is_attributed'],axis=1),train_gbc['is_attributed'], test_size=0.2, random_state=42)

In [22]:
train_gbc.head()

,channel,ip,channel_count_app,app,app_count_channel,is_attributed
0,213,326487,1329,19,1335,1
1,477,114414,179,2,583,0
2,478,86767,35,20,68,0
3,343,282934,117,29,393,0
4,130,121384,32,3,945,0


In [23]:
# X_train = train_gbc[['channel','ip','channel_count_app','app','app_count_channel']]
# y_train = train_gbc[['is_attributed']]

In [24]:
# X_test = valid_gbc[['channel','ip','channel_count_app','app','app_count_channel']]
# y_test = valid_gbc[['is_attributed']]

In [25]:
gbc_full = GradientBoostingClassifier()
gbc_reduced = GradientBoostingClassifier()

In [26]:
x_full_train.head()

,ip,app,device,os,channel,day,hour,minute,second,ip_app_channel_var_day,...,ip_nunique_app,ip_day_nunique_hour,ip_app_nunique_os,ip_nunique_device,app_nunique_channel,ip_device_os_nunique_app,ip_device_os_cumcount_app,ip_cumcount_app,ip_cumcount_os,ip_day_channel_var_hour
6599,203210,29,1,34,213,7.0,10.0,47.0,40.0,0.0,...,1,1,1,1,8,1,0,0,0,0.0
5948,198350,18,1,13,107,9.0,8.0,55.0,1.0,0.0,...,1,1,1,1,9,1,0,0,0,0.0
1420,129027,10,1,11,113,7.0,7.0,19.0,18.0,0.0,...,1,1,1,1,6,1,0,0,0,0.0
2389,36339,3,1,17,280,8.0,16.0,33.0,12.0,0.0,...,3,1,1,1,31,1,0,1,1,0.0
2404,277726,29,1,17,213,8.0,5.0,9.0,2.0,0.0,...,1,1,1,1,8,1,0,0,0,0.0


In [27]:
len(train_fe.columns)

31

In [28]:
gbc_full.fit(x_full_train, y_full_train)
roc_auc_score(gbc_full.predict(x_full_test),y_full_test)

0.9255315388557996

In [29]:
gbc_reduced.fit(x_reduced_train, y_reduced_train)
roc_auc_score(gbc_reduced.predict(x_reduced_test),y_reduced_test)

0.9159187888175178

In [30]:
roc_auc_score(gbc_full.predict(x_full_test),y_full_test)

0.9255315388557996

In [31]:
train_lgb = train_fe[['ip',
 'os',
 'channel_count_app',
 'minute',
 'app',
 'second',
 'channel',
 'hour',
 'app_count_channel',
 'app_AvgViewPerDistinct_ip',
 'is_attributed']]
valid_lgb = train_fe[['ip',
 'os',
 'channel_count_app',
 'minute',
 'app',
 'second',
 'channel',
 'hour',
 'app_count_channel',
 'app_AvgViewPerDistinct_ip',
 'is_attributed']]

In [32]:
X_train = train_lgb.drop(['is_attributed'],axis=1)
y_train = train_lgb[['is_attributed']]
X_test = valid_lgb.drop(['is_attributed'],axis=1)
y_test = valid_lgb[['is_attributed']]

In [33]:
lgb_full = LGBMClassifier()
lgb_reduced = LGBMClassifier()

In [ ]:
roc_auc_score(lgb_full.fit(X_train.values, y_train).predict(X_test.values), y_test)
roc_auc_score(lgb_reduced.fit(X_train.values, y_train).predict(X_test.values), y_test)

In [36]:
train_rfc = train_fe[['app',
 'channel_count_app',
 'app_nunique_channel',
 'app_count_channel',
 'ip',
 'channel',
 'app_AvgViewPerDistinct_ip',
 'second',
 'minute',
 'os',
 'is_attributed']]
valid_rfc = train_fe[['app',
 'channel_count_app',
 'app_nunique_channel',
 'app_count_channel',
 'ip',
 'channel',
 'app_AvgViewPerDistinct_ip',
 'second',
 'minute',
 'os',
 'is_attributed']]

In [37]:
X_train = train_rfc.drop(['is_attributed'],axis=1)
y_train = train_rfc[['is_attributed']]
X_test = valid_rfc.drop(['is_attributed'],axis=1)
y_test = valid_rfc[['is_attributed']]

In [47]:
roc_auc_score(gbc_full.predict(X),y)

0.5014488225344459